In [4]:
import pandas as pd
import numpy as np
import pickle
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin
from catboost import CatBoostRegressor

# ----------------------------------------
# Wrapper de CatBoost que detecta automáticamente
# las columnas categóricas (tipo object) y las pasa a fit.
class CatBoostWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, iterations=100, learning_rate=0.1, depth=6, **kwargs):
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.depth = depth
        self.kwargs = kwargs
        self.model = None

    def fit(self, X, y):
        # Detecta índices de columnas categóricas (dtype object)
        cat_features = np.where(X.dtypes == 'object')[0].tolist()
        # Asegurarse de que no hay floats en columnas categóricas
        X = X.copy()
        for col_idx in cat_features:
            col_name = X.columns[col_idx]
            X[col_name] = X[col_name].astype(str)

        self.model = CatBoostRegressor(
            iterations=self.iterations,
            learning_rate=self.learning_rate,
            depth=self.depth,
            verbose=0,
            **self.kwargs
        )
        self.model.fit(X, y, cat_features=cat_features)
        return self

    def predict(self, X):
        return self.model.predict(X)

# ----------------------------------------
# Función para cargar el dataset desde un archivo CSV
def load_data(filepath):
    df = pd.read_csv(filepath)
    return df

# ----------------------------------------
# Función de preprocesamiento mínimo
def preprocess_data(df):
    # Se eliminan columnas descartables
    cols_to_drop = ['provincia', 'PrecioAnterior', 'Enlace', 'ascensor', 'localizacion', 'planta', 'tags', 'descripcion']
    df = df.drop(columns=cols_to_drop, errors='ignore')
    
    df['TipoVivienda'] = df['titulo'].str.strip().str.split().str[0]
    df = df.drop(columns='titulo')
    # En este ejemplo se dejan las demás columnas tal cual,
    # por lo que las variables categóricas se mantienen en su forma original.
    # Puedes, si lo consideras, hacer otros ajustes mínimos.
    df = df.rename(columns={"baños": "banos"})
    # Eliminar filas con valores nulos (opcional, según la situación)
    df = df.fillna("N/A")
    df = df.dropna()
    return df

# ----------------------------------------
# Función para separar la variable objetivo y las features
def separate_target(df):
    y = df['PrecioActual']
    X = df.drop(columns=['PrecioActual'])
    cat_features = np.where(X.dtypes == 'object')[0].tolist()
    X = X.copy()
    for col_idx in cat_features:
        col_name = X.columns[col_idx]
        X[col_name] = X[col_name].astype(str)
    return X, y

# ----------------------------------------
# Función para optimizar hiperparámetros usando GridSearchCV
def tune_model(estimator, param_grid, X_train, y_train):
    grid = GridSearchCV(estimator, param_grid=param_grid, 
                        cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_estimator = grid.best_estimator_
    best_cv_mse = -grid.best_score_
    print("Mejores parámetros:", grid.best_params_)
    print("Mejor MSE en CV:", best_cv_mse)
    return best_estimator, best_cv_mse

# ----------------------------------------
# Función para evaluar el modelo en el conjunto de test
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, mae, r2

# ----------------------------------------
# Función principal
def main():
    # 1. Cargar datos
    filepath = './Datos.csv'  # Asegúrate de ajustar la ruta
    df = load_data(filepath)
    
    # 2. Preprocesamiento (mínimo, sin transformar variables categóricas)
    df = preprocess_data(df)
    df.to_csv('Datos_preprocesados.csv', index=False)
    # 3. Separar variable objetivo (PrecioActual) y variables predictoras
    X, y = separate_target(df)
    
    # 4. División en conjuntos de entrenamiento y test (80% - 20%)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    
    # 5. Definir el modelo (solo CatBoost) con el wrapper
    catboost_model = CatBoostWrapper(random_state=42)
    
    # 6. Definir grid de hiperparámetros para CatBoost
    param_grid = {
        'iterations': [200],
        'learning_rate': [0.1],
        'depth': [7]
    }
    
    # 7. Optimización con GridSearchCV
    best_model, cv_mse = tune_model(catboost_model, param_grid, X_train, y_train)
    
    # 8. Evaluación en el conjunto de test
    test_mse, test_mae, test_r2 = evaluate_model(best_model, X_test, y_test)
    print("\nEvaluación en test:")
    print(f"MSE:  {test_mse:.3f}")
    print(f"MAE:  {test_mae:.3f}")
    print(f"R²:   {test_r2:.3f}")
    
    # 8. Guardar el modelo optimizado en un archivo pickle
    with open('best_catboost_model.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    print("\nModelo guardado en 'best_catboost_model.pkl'.")

if __name__ == "__main__":
    main()


Mejores parámetros: {'depth': 7, 'iterations': 200, 'learning_rate': 0.1}
Mejor MSE en CV: 285898538500.78534

Evaluación en test:
MSE:  246904840418.552
MAE:  235352.851
R²:   0.842

Modelo guardado en 'best_catboost_model.pkl'.


In [3]:
filepath = './Datos.csv'
df = pd.read_csv(filepath)
df.info()
cols_to_drop = ['provincia', 'PrecioAnterior', 'Enlace', 'ascensor', 'localizacion', 'planta', 'tags', 'descripcion']
df = df.drop(columns=cols_to_drop, errors='ignore')

df['TipoVivienda'] = df['titulo'].str.strip().str.split().str[0]
df = df.drop(columns='titulo')

# En este ejemplo se dejan las demás columnas tal cual,
# por lo que las variables categóricas se mantienen en su forma original.
# Puedes, si lo consideras, hacer otros ajustes mínimos.

# Eliminar filas con valores nulos (opcional, según la situación)
df = df.fillna("N/A")
df = df.dropna()
df
df['TipoVivienda'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11826 entries, 0 to 11825
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   provincia       11826 non-null  object 
 1   zona            11826 non-null  object 
 2   titulo          11826 non-null  object 
 3   PrecioActual    11826 non-null  int64  
 4   PrecioAnterior  11826 non-null  int64  
 5   metros          11826 non-null  int64  
 6   habitaciones    11460 non-null  float64
 7   ascensor        11033 non-null  object 
 8   localizacion    10730 non-null  object 
 9   planta          10601 non-null  object 
 10  baños           11826 non-null  int64  
 11  tags            11664 non-null  object 
 12  descripcion     11761 non-null  object 
 13  Enlace          11826 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 1.3+ MB


TipoVivienda
Piso       9551
Ático       782
Dúplex      435
Chalet      398
Estudio     339
Casa        321
Name: count, dtype: int64